In [10]:
# %%
# Script written by Maximilien Van Migem 12 June 2025 Ghent
# This script converts the original dataset in to the BIDS format
import os.path as op
from pprint import pprint
import shutil
import glob
import mne
import mne_bids
import datetime
import numpy as np
import pandas as pd

In [5]:
# %%
# Data paths
data_path = r'C:\Users\mvmigem\Documents\data\project_1'
raw_dir_path = op.join(data_path,'raw_data')
event_dir_path = op.join(data_path,'preprocessed','mastoid_ref_csv','events')
bids_root = op.join(data_path, 'project_1-bids-data')

# Subjects
subjects_list = np.arange(1,27)
subject_ids = subjects_list[subjects_list != 20]

In [16]:
if op.exists(bids_root):
    shutil.rmtree(bids_root)

In [15]:
# %%
# Loop to create a list of raws
raw_list = []
bids_list =[]
event_list = []
event_id_list = []
event_metadata_list = []

gender_mapping = {'Other':0,'Male':1,'Female':2}
hand_mapping = {'right':1,'left':2,'ambi':3}

for sub in subject_ids:
    
    bid_sub_id = f'{sub:02}'
    # Load eeg
    raw_path = op.join(raw_dir_path,f'sub_{sub}','eeg','*.bdf')
    raw_fname = glob.glob(raw_path) [0]
    raw = mne.io.read_raw_bdf(raw_fname, eog = (), preload=False, verbose=50)
    epo_path = op.join(data_path,f'preprocessed/mastoid_ref/main_eventset_mastoidref_{sub:02}-epo.fif')
    epo_path = op.join(data_path,f'preprocessed/mastoid_ref/main_eventset_mastoidref_{sub:02}-epo.fif')
    epochs = mne.read_epochs(epo_path)
    # Load behaviour
    behav_path = raw_dir_path + f'/sub_{sub}/behav/predatt_participant_{sub}.csv'
    behav = pd.read_csv(behav_path)
    # Load events 
    events = epochs.events
    events_id = epochs.event_id

    events_path = op.join(event_dir_path,f'events_{sub:02}.csv')
    events_metadata = pd.read_csv(events_path)
    events_metadata = events_metadata.drop(columns=['sample','event_codes'])
    


    event_list.append(events)
    event_id_list.append(events_id)
    event_metadata_list.append(events_metadata)
    raw.set_annotations(None)
    # Add to list
    raw_list.append(raw)
    # specify power line frequency, and ref as required by BIDS
    raw.info['line_freq'] = 50
    # specify sub info
    sub_age = int(behav['age'].iloc[0])
    sub_birthday = datetime.date(2023-sub_age,1,1)
    sub_gender = behav['gender'].iloc[0]
    mapped_gender = gender_mapping[sub_gender]
    sub_hand = behav['handed'].iloc[0]
    mapped_hand = hand_mapping[sub_hand]
    
    subject_info = {'id':int(sub),'his_id':f'sub-{sub:02}','birthday':sub_birthday,'sex':mapped_gender,'hand':mapped_hand}
    raw.info['subject_info'] = subject_info
    # specify task
    task ='targetdetection'
    # Create BIDS path
    bids_path = mne_bids.BIDSPath(subject=bid_sub_id,task = task, datatype='eeg', root=bids_root)
    # Add to list
    bids_list.append(bids_path)



Reading C:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_01-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4189 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_02-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4196 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_03-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 32 columns
4188 matching events found
N

In [19]:
daysback_min, daysback_max = mne_bids.get_anonymization_daysback(raw_list)

metadata_discription = {"position":"spatial position of the stimulus 1,2,3 or 4 corrsponding to the top left, top right, bottom right or bottom left respectively. 9 signals trial start",
                        "sequence":"the order of presentation of the stimulus. 9 signals the trial start",
                        "attention":"whether attention is directed towards the peripheral stimulus (attended) or towards the fixation (unattended)",
                        "expectation":" whether the trial follows a regular or odd presentation pattern",
                        "setup":"the position on which the second stimulus of the trial was presented in the upper visual field",
                        "setdown":"the position on which the second stimulus of the trial was presented in the lower visual field",
                        "subject":"subject number",
                        "selected_electrode":"the electrode that showed the largest C1 amplitude in the localiser"}

for raw, bids_path, event,event_id,event_metadata in zip(raw_list, bids_list,event_list,event_id_list,event_metadata_list):

    mne_bids.write_raw_bids(
        raw, bids_path, 
        events=event,
        event_id=event_id,
        event_metadata=event_metadata,
        extra_columns_descriptions= metadata_discription,
        anonymize=dict(daysback=daysback_min + 3652), 
        overwrite=True
    )

Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\README'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.json'...
Used Annotations descriptions: [np.str_('pos1/seq2'), np.str_('pos1/seq4'), np.str_('pos2/seq3'), np.str_('pos3/seq2'), np.str_('pos3/seq4'), np.str_('pos4/seq1'), np.str_('start_trial')]
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-01\eeg\sub-01_task-targetdetection_events.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-01\eeg\sub-01_task-targetdetection_events.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\dataset_description.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-01\eeg\sub-01_task-targetdetection_eeg.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: Omitted 3741 annotation(s) that were outside data range.
  mne_bids.write_raw_bids(
C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: EDF/EDF+/BDF files contain two fields for recording dates.Due to file format limitations, one of these fields only supports 2-digit years. The date for that field will be set to 85 (i.e., 1985), the earliest possible date. The true anonymized date is stored in the scans.tsv file.
  mne_bids.write_raw_bids(
C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  mne_bids.write_raw_bids(


Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.json'...
Used Annotations descriptions: [np.str_('pos1/seq2'), np.str_('pos1/seq4'), np.str_('pos2/seq1'), np.str_('pos2/seq3'), np.str_('pos3/seq2'), np.str_('pos3/seq4'), np.str_('pos4/seq1'), np.str_('pos4/seq3'), np.str_('start_trial')]
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\eeg\sub-02_task-targetdetection_events.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\eeg\sub-02_task-targetdetection_events.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\dataset_description.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\eeg\sub-02_task-targetdetection_eeg.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\eeg\sub-02_task-targetdetection

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: EDF/EDF+/BDF files contain two fields for recording dates.Due to file format limitations, one of these fields only supports 2-digit years. The date for that field will be set to 85 (i.e., 1985), the earliest possible date. The true anonymized date is stored in the scans.tsv file.
  mne_bids.write_raw_bids(


Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\sub-02_scans.tsv'...
Wrote C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-02\sub-02_scans.tsv entry with eeg\sub-02_task-targetdetection_eeg.bdf.
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.json'...
Used Annotations descriptions: [np.str_('pos1/seq2'), np.str_('pos1/seq4'), np.str_('pos2/seq1'), np.str_('pos2/seq3'), np.str_('pos3/seq2'), np.str_('pos3/seq4'), np.str_('pos4/seq1'), np.str_('pos4/seq3'), np.str_('start_trial')]
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-03\eeg\sub-03_task-targetdetection_events.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-03\eeg\sub-03_task-targetdetection_events.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\dataset_descriptio

...

Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-24\sub-24_scans.tsv'...
Wrote C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-24\sub-24_scans.tsv entry with eeg\sub-24_task-targetdetection_eeg.bdf.
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.json'...
Used Annotations descriptions: [np.str_('pos1/seq'), np.str_('pos1/seq3'), np.str_('pos2/seq2'), np.str_('pos2/seq4'), np.str_('pos3/seq1'), np.str_('pos3/seq3'), np.str_('pos4/seq2'), np.str_('pos4/seq4'), np.str_('start_trial')]
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-25\eeg\sub-25_task-targetdetection_events.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-25\eeg\sub-25_task-targetdetection_events.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\dataset_description

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: EDF/EDF+/BDF files contain two fields for recording dates.Due to file format limitations, one of these fields only supports 2-digit years. The date for that field will be set to 85 (i.e., 1985), the earliest possible date. The true anonymized date is stored in the scans.tsv file.
  mne_bids.write_raw_bids(


Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-25\sub-25_scans.tsv'...
Wrote C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-25\sub-25_scans.tsv entry with eeg\sub-25_task-targetdetection_eeg.bdf.
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\participants.json'...
Used Annotations descriptions: [np.str_('pos1/seq'), np.str_('pos1/seq3'), np.str_('pos2/seq2'), np.str_('pos2/seq4'), np.str_('pos3/seq1'), np.str_('pos3/seq3'), np.str_('pos4/seq2'), np.str_('pos4/seq4'), np.str_('start_trial')]
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-26\eeg\sub-26_task-targetdetection_events.tsv'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-26\eeg\sub-26_task-targetdetection_events.json'...
Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\dataset_description

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_18912\3419323190.py:14: RuntimeWarning: EDF/EDF+/BDF files contain two fields for recording dates.Due to file format limitations, one of these fields only supports 2-digit years. The date for that field will be set to 85 (i.e., 1985), the earliest possible date. The true anonymized date is stored in the scans.tsv file.
  mne_bids.write_raw_bids(


Writing 'C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-26\sub-26_scans.tsv'...
Wrote C:\Users\mvmigem\Documents\data\project_1\project_1-bids-data\sub-26\sub-26_scans.tsv entry with eeg\sub-26_task-targetdetection_eeg.bdf.
